# Imports

In [33]:
from datascience import *
from IPython.display import *
from ipywidgets import *
import pandas as pd
import numpy as np
import plotly.express as px;
import scipy.constants as constant 
import math
from ipywidgets import *
import re
import json
import dataframe_image as dfi
from typing import *

# General Utilities

In [55]:
# Dict reader
def read(fp: str, full_fp: bool = False) -> dict:
    """
    ### Creates a dictionary of the given JSON file

    Parameters
    ----------
    fp : `str`
        The file path of the JSON file
        such that file is named `data/json/<fp>.json`

    full_fp : `bool`
        Whether the filepath is the full filepath or not

    Returns
    -------
    `dict`
        A dictionary of the JSON file

    Examples
    --------
    >>> read("example.json")
    {'a': 1, 'b': 2, 'c': 3}
    """
    if not full_fp:
        fp = f"{fp}.json"
    return json.load(open(fp, "r"))

def show(*args, tags = []):
    """Pretty Display"""
    assert (tags == []) or (type(tags[0]) == str), "tags must contain strings"
    for i in args:
        if type(i) != str:
            i = str(i)
        for tag in tags:
            i = f"<{tag}>{i}</{tag}>"
        display(Markdown(i))

# Get and Clean the Data

In [60]:
def get_data() -> pd.DataFrame:
    """
    Get data from the Google Sheet and add the section column.
    
    Parameters
    ----------
    
    None
    
    Returns
    -------
    pd.DataFrame
        Data from the database with the section column.
    """
    df = fetch_data()
    df = add_section_column(df)
    df = rename_columns(df)
    return df

def fetch_data() -> pd.DataFrame:
    """
    Fetches data via google sheets from a fixed url.
    
    Parameters
    ----------
    
    None
    
    Returns
    -------
    
    data : pd.DataFrame
        The data from the google sheet
    """
    url = "https://docs.google.com/spreadsheets/d/1RH96Rjr_bC5bcpd3DqY5shsuEMtxlk4NN39pUYrGeNA/edit#gid=0"
    csv_url = re.sub("/edit#gid=", "/export?format=csv&gid=", url)
    data = pd.read_csv(csv_url)
    return data

def add_section_column(data : pd.DataFrame) -> pd.DataFrame:
    """
    Adds a column to the data that contains the section number.
    
    Parameters
    ----------
    
    data : pd.DataFrame
        The data from the google sheet
    
    Returns
    -------
    
    data : pd.DataFrame
        The data with the section column added and columns reordered.
    """
    section_dict = read("teams")
    section_from_team = lambda team: section_dict[str(team)]
    data["Section"] = data["Team"].apply(section_from_team)
    data = data[["Team", "Section", "Mass", "Angle", "ShearForce", "AdhesiveForce"]]
    return data

def rename_columns(data : pd.DataFrame) -> pd.DataFrame:
    """
    Renames the columns of the data to be more readable.
    
    Parameters
    ----------
    
    data : pd.DataFrame
        The data from the google sheet
    
    Returns
    -------
    
    data : pd.DataFrame
        The data with the columns renamed.
    """
    data = data.rename(columns={"Mass": "Mass (g)", "Angle": "Angle (degrees)", "ShearForce": "Shear Force (N)", "AdhesiveForce": "Adhesive Force (N)"})
    return data

def section_from_team(team : int) -> int:
    """ 
    Gets the section number for a given team.
    
    Parameters
    ----------
    
    team : int
        The team number to get the section for.
    
    Returns
    -------
    
    int : The section number for the given team.
    
    """
    sections = read("teams")
    return sections[str(team)]

def get_data_for_team(team : int) -> pd.DataFrame:
    """
    Gets data for a specific team.
    
    Parameters
    ----------
    
    team : int
        The team number to get data for.
    
    Returns
    -------
    
    team_data : pd.DataFrame
        The data for the given team.
    """
    team = team
    section = section_from_team(team)
    data = get_data()
    team_data = data[data["Team"] == team]
    return team_data

def select_team():
    """
    Use this function to select a team and get the data for that team.
    Utilizes the interact function from ipywidgets.
    
    Parameters
    ----------
    
    None
    
    Returns
    -------
    
    pd.DataFrame : The data filtered for the selected team.
    
    """
    show("Select a team to get data for.", tags=["h2"])
    show("You can also use the arrow keys to change the team number, or simply type it into the box.", tags=["h3"])
    show("Note: If typing your team number, click out of the 'Team:' box once you are done", tags=["h3", "blockquote"])
    w = (interact(get_data_for_team, team=BoundedIntText(value=1, min=1, max=35, step=1, description="Team:")))
    return w.widget.result

def get_info(df : pd.DataFrame) -> Tuple[int, int]:
    """
    Retrieves the team and section number from the data.
    
    Parameters
    ----------
    
    df : pd.DataFrame
        The data for a spcecific team.
    
    Returns
    -------
    
    tuple : A tuple containing the team number and section number.
    """
    section = df["Section"].iloc[0]
    team = df["Team"].iloc[0]
    return (team, section)

data = select_team()
team, section = get_info(data)

<h2>Select a team to get data for.</h2>

<h3>You can also use the arrow keys to change the team number, or simply type it into the box.</h3>

<blockquote><h3>Note: If typing your team number, click out of the 'Team:' box once you are done</h3></blockquote>

interactive(children=(BoundedIntText(value=1, description='Team:', max=35, min=1), Output()), _dom_classes=('w…